In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

/home/samuel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv("../final_with_labels.csv")
data_matrix = data.as_matrix()

In [3]:
data.head()

,ID,pubcsnum,mar_stat,sex,primsite,histo3v,beho3v,grade,dx_conf,csexten,...,maligcount,benbordcount,year_dx,nhiade,codpub,srv_time_mon,srv_time_mon_flag,surv_1,surv_2,surv_5
0,1,7011365,9,2,C184,8210,3,2,1,200,...,4,0,2010,7,0,71,1,1,1,1
1,2,7014780,5,2,C209,8140,3,2,1,200,...,2,0,2007,6,21040,36,1,1,1,0
2,3,7017183,2,1,C186,8140,2,9,1,50,...,3,0,2012,6,21050,37,1,1,1,0
3,4,7017183,2,1,C209,8263,3,2,1,455,...,3,0,2012,6,21050,37,1,1,1,0
4,5,7018423,2,1,C186,8263,3,3,1,400,...,3,0,2011,2,0,52,1,1,1,1


In [4]:
data_matrix.shape

(46785, 33)

In [5]:
data_matrix[0]

array([1, 7011365, 9, 2, 'C184', 8210, 3, 2, 1, 200, 0, 0, 3.0, 3.0, 0.0,
       50, 0, 1, 1502, 93, 0, 5, 30, 4, 0, 2010, 7, 0, 71, 1, 1, 1, 1],
      dtype=object)

# Preprocessing

In [6]:
Y1 = data_matrix[:,-3]
Y2 = data_matrix[:,-2]
Y5 = data_matrix[:,-1]

In [7]:
X = data_matrix[:,2:24]
X_cat = X[:,:-5]
# primary site is only categorical variable with string labels
X_primsite = X[:,2]
X_cont = X[:,-5:]
# X_cat contains only categorical variables with integer labels
X_cat = np.delete(X_cat, [2], axis=1)

In [8]:
# Fill missing values in three eval columns with 9: unknown
X_cat[:,9] = np.array([9 if np.isnan(x) else x for x in X_cat[:,9]])
X_cat[:,10] = np.array([9 if np.isnan(x) else x for x in X_cat[:,10]])
X_cat[:,11] = np.array([9 if np.isnan(x) else x for x in X_cat[:,11]])

In [9]:
# one-hot encode primary site
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
X_primsite_1hot = encoder.fit_transform(X_primsite)
encoder.classes_

array(['C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187',
       'C188', 'C189', 'C199', 'C209', 'C260'], dtype='<U4')

In [10]:
# one-hot encode remaining categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
X_cat_1hot = encoder.fit_transform(X_cat)

In [11]:
# combine all categorical variables
X_cat = np.hstack((X_primsite_1hot, X_cat_1hot))

The continuous variables in order: age_dx, eod10_pn, eod10_ne, cstumsiz, maligcount. Age and maligcount do not reuqire imputation, and eod10_ne (nodes examined) will not be used as a variable.


Here we will impute values for eod10_pn and cstumsiz. We will implement multivariate imputation by chained equations (MICE) on these two continuous variables. Their indices are and values which need to be imputed are:

1: eod10_pn (95, 97, 98, 99)<br>
3: cstumsiz (998, 999)<br>

Additionally, we will replace the values (991, 992, 993, 994, 995) in cstumsiz with (1, 2, 3, 4, 5) respectively.


### Note: The above should be re-verified for each different dataset. This is for colorectal cancer in Hispanic patients after 2004.

In [12]:
# implements MICE imputation with linear regression model on eod10_pn and cstumsiz
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression

NUM_CYCLES=10

missing_values_pn = (95, 97, 98, 99)
missing_values_tumsiz = (998, 999)

pn = np.array(X_cont[:,1]).reshape(-1,1)
tumsiz = np.array(X_cont[:,3]).reshape(-1,1)

# non-MICE imputation
for i, size in enumerate(tumsiz):
    if size not in (990, 991, 992, 993, 994, 995):
        continue
    elif size == 990:
        tumsiz[i] = 0
    elif size == 991:
        tumsiz[i] = 1
    elif size == 992:
        tumsiz[i] = 2
    elif size == 993:
        tumsiz[i] = 3
    elif size == 994:
        tumsiz[i] = 4
    elif size == 995:
        tumsiz[i] = 5


# set missing values to -1
for i, value in enumerate(pn):
    if value in missing_values_pn:
        pn[i] = -1
for i, value in enumerate(tumsiz):
    if value in missing_values_tumsiz:
        tumsiz[i] = -1

# indices for valid and imputation-needed positions in pn and tumsiz
idx_pn_missing = np.where(pn == -1)[0]
idx_pn_valid = np.where(pn != -1)[0]
idx_tumsiz_missing = np.where(tumsiz == -1)[0]
idx_tumsiz_valid = np.where(tumsiz != -1)[0]
    
# set missing values to mean
# copy = false modifies array in place
imp_pn = Imputer(missing_values = -1, copy=False)
imp_pn.fit_transform(pn)
imp_tumsiz = Imputer(missing_values = -1, copy=False)
imp_tumsiz.fit_transform(tumsiz)


for i in np.arange(NUM_CYCLES):
    print("Cycle " + str(i+1))
    
    # fit pn
    X_ind = np.hstack((X_cat[idx_pn_valid], tumsiz[idx_pn_valid]))
    X_rep = np.hstack((X_cat[idx_pn_missing], tumsiz[idx_pn_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, pn[idx_pn_valid])
    pn[idx_pn_missing] = lin_reg.predict(X_rep)
    pn = np.maximum(pn, 0)
    pn = np.minimum(pn, 100)
    
    # fit tumsiz
    X_ind = np.hstack((X_cat[idx_tumsiz_valid], pn[idx_tumsiz_valid]))
    X_rep = np.hstack((X_cat[idx_tumsiz_missing], pn[idx_tumsiz_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, tumsiz[idx_tumsiz_valid])
    tumsiz[idx_tumsiz_missing] = lin_reg.predict(X_rep)
    tumsiz = np.maximum(tumsiz, 0)
    tumsiz = np.minimum(tumsiz, 990)


Cycle 1
Cycle 2
Cycle 3
Cycle 4
Cycle 5
Cycle 6
Cycle 7
Cycle 8
Cycle 9
Cycle 10


In [13]:
pn = pn.reshape(-1,).astype(int)
Counter(pn)

Counter({0: 30142,
         1: 4741,
         2: 3102,
         3: 2328,
         4: 1530,
         5: 1088,
         6: 791,
         7: 550,
         8: 432,
         9: 340,
         10: 287,
         11: 220,
         12: 189,
         13: 143,
         14: 128,
         15: 125,
         16: 80,
         17: 69,
         18: 53,
         19: 42,
         20: 48,
         21: 25,
         22: 38,
         23: 23,
         24: 24,
         25: 19,
         26: 10,
         27: 13,
         28: 14,
         29: 9,
         30: 4,
         31: 3,
         32: 6,
         33: 4,
         34: 5,
         35: 5,
         36: 2,
         37: 4,
         38: 2,
         39: 3,
         40: 2,
         41: 1,
         42: 1,
         44: 2,
         59: 1,
         68: 1,
         71: 1,
         77: 1,
         90: 1,
         100: 133})

In [14]:
print(np.mean(pn[idx_pn_valid]))
print(np.std(pn[idx_pn_valid]))
print(np.mean(pn[idx_pn_missing]))
print(np.std(pn[idx_pn_missing]))

1.9226782686586263
3.956180991059097
1.4276053072265877
9.25536956566678


In [ ]:
print(np.mean(tumsiz[idx_tumsiz_valid]))
print(np.std(tumsiz[idx_tumsiz_valid]))
print(np.mean(tumsiz[idx_tumsiz_missing]))
print(np.std(tumsiz[idx_tumsiz_missing]))

In [ ]:
from collections import Counter
Counter(X_cont[:,4])

# Training

For hispanic colrect, there doesn't seem to be any categorical values in malig.

In [15]:
age_dx = np.array(X_cont[:,0])
malig = np.array(X_cont[:,4])
X_all = np.column_stack((X_cat, age_dx, pn, malig, tumsiz))
print(X_all.shape)

(46785, 313)


In [16]:
# shuffle
np.random.seed(97)
idx = np.random.permutation(len(X_all))
X = X_all[idx]
Y = Y5[idx]

TEST_SET_SIZE = 6000

X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)

In [61]:
np.save('../X_all.npy', X_all)
np.save('../Y5.npy', Y5)
np.save('../Y2.npy', Y2)
np.save('../Y1.npy', Y1)

In [ ]:
len(np.where(Y==1)[0]) / len(Y)

# Feature Scaling

In [17]:
# feature scaling: scale features based on training data only
from sklearn.preprocessing import StandardScaler, MinMaxScaler
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.fit_transform(X_test)
#mm_scaler = MinMaxScaler()
#X_train = mm_scaler.fit_transform(X_train)

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
# min_sample_split: 300,400
# min_samples_leaf: 200
# max_depth: 130
# min_weight_fraction_leaf: .01
param_grid = [{'min_samples_leaf':[300,400,500]}]
tree_clf_reg = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_clf_reg, param_grid, cv=5, scoring="roc_auc", verbose=1)
grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(grid_search.best_params_))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, Y_train.astype(int))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = knn_clf.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = [{'penalty':['l2'], 'C':[0.1]}]
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=5)
grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(grid_search.best_params_))

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        max_samples=1000, bootstrap=True, n_jobs=-1,
)

bag_clf.fit(X_train, Y_train)

In [ ]:
# probably add cross-validation?

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = bag_clf.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, Y_train)

In [ ]:
y_pred_rf = rnd_clf.predict(X_test)
accuracy_score(y_pred_rf, Y_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

svm_clf = Pipeline((
                        ("scaler", StandardScaler()),
                        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
))
svm_clf.fit(X_train, Y_train)

In [ ]:
accuracy_score(svm_clf.predict(X_test), Y_test)
accuracy_score(svm_clf.predict(X_train), Y_train)

## Neural Network

In [33]:
import tensorflow as tf

n_inputs = X.shape[1]
n_hidden1 = 300
n_hidden2 = 300
n_hidden3 = 300
n_hidden4 = 812
n_hidden5 = 812
n_outputs = 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
# with tf.name_scope("dnn"):
#     hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
#                            activation=tf.nn.relu)
#     hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
#                            activation=tf.nn.relu)
#     hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3",
#                            activation=tf.nn.relu)
#     hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4",
#                            activation=tf.nn.relu)
#     hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5",
#                            activation=tf.nn.relu)
#     logits = neuron_layer(hidden5, n_outputs, name="outputs")

In [56]:
tf.reset_default_graph()

In [58]:
from tensorflow.contrib.layers import fully_connected

def leaky_relu(z, name=None):
    return tf.maximum(0.01*z, z, name=name)

with tf.name_scope("dnn"):

    hidden1 = fully_connected(X, n_hidden1, activation_fn=leaky_relu, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, activation_fn=leaky_relu, scope="hidden2")
    hidden3 = fully_connected(hidden2, n_hidden3, activation_fn=leaky_relu, scope="hidden3")
    logits = fully_connected(hidden3, n_outputs, activation_fn=None, scope="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

ValueError: Variable hidden1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/samuel/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 246, in variable
    use_resource=use_resource)
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 183, in func_with_args
    return func(*args, **current_args)
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 291, in model_variable
    use_resource=use_resource)


In [53]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

ValueError: Tensor("loss_6/Const:0", shape=(1,), dtype=int32) must be from the same graph as Tensor("SparseSoftmaxCrossEntropyWithLogits_5/SparseSoftmaxCrossEntropyWithLogits:0", shape=(?,), dtype=float32).

In [30]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [31]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [38]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs=15
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train)//batch_size):
            start_idx = iteration*batch_size
            end_idx = start_idx+batch_size
            X_batch, y_batch = X_train[start_idx:end_idx], Y_train[start_idx:end_idx]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: Y_test})
        print(epoch, "train accuracy:", acc_train, "Test accuracy:", acc_test)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

InvalidArgumentError: You must feed a value for placeholder tensor 'X' with dtype float and shape [?,313]
	 [[Node: X = Placeholder[dtype=DT_FLOAT, shape=[?,313], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'X', defined at:
  File "/home/samuel/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/samuel/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/samuel/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, 

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = logits.eval(feed_dict={X: X_test})
    Y_pred = np.argmax(Z, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred)))
confusion_matrix(Y_test.astype(int), Y_pred)